---
format:
  html:
    embed-resources: true
    code-fold: true
---

## Data Cleaning

##### __Links to source code and raw data__
Link to source of cleaning data code (Python and R) on Github: [Source Code](https://github.com/anly501/dsan-5000-project-shawnhxu/tree/main/codes/02-data-cleaning)

Link to raw data on Github: [Raw Data](https://github.com/anly501/dsan-5000-project-shawnhxu/tree/main/data/01-raw-riot-data) <br>
*The retrieval of this data was explained in "Data Gathering" tab.*

Link to cleaned data on Github: [Cleaned Data](https://github.com/anly501/dsan-5000-project-shawnhxu/tree/main/data/02-cleaned_riot_data)

## __Cleaning Process__
All the raw data that I retrieved from the Riot API can be found in the link above^. And details about how I retrieved the data along with the code of how I did the retrieval can be found in the "Data Gathering" tab. 

![Riot Image [Source](https://www.riotgames.com/en/press)](images/riot.png)

<br>

```python
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
```

__Python Cleaning__<br>
One of the first things after retrieving the data is of course importing the data set from a .csv file. In my case, since I retrieved many matches of different recordable data from each rank division I had seven different data sets of match data to import. This was simply done using the package from Pandas to read .csv files and importing them into Python. Also, it is worth noting that the code that I used to import uses the a relative path that is the same as seen on github. The code for importing:

In [ ]:
#file path to data
iron_path = '../../data/raw-riot-data/league data/league_iron_data.csv'
bronze_path = '../../data/raw-riot-data/league data/league_bronze_data.csv'
silver_path = '../../data/raw-riot-data/league data/league_silver_data.csv'
gold_path = '../../data/raw-riot-data/league data/league_gold_data.csv'
platinum_path = '../../data/raw-riot-data/league data/league_platinum_data.csv'
emerald_path = '../../data/raw-riot-data/league data/league_emerald_data.csv'
diamond_path = '../../data/raw-riot-data/league data/league_diamond_data.csv'

#import all .csv files of each ranked division match data
#all blank entries in the dataset will be set as NaN by pandas
iron_df = pd.read_csv(iron_path)
bronze_df = pd.read_csv(bronze_path)
silver_df = pd.read_csv(silver_path)
gold_df= pd.read_csv(gold_path)
platinum_df= pd.read_csv(platinum_path)
emerald_df= pd.read_csv(emerald_path)
diamond_df = pd.read_csv(diamond_path)

After importing the .csv files and putting them in properly named variables, I actually first examined the original .csv files before deciding on what to clean first. What I noticed off the bat is that each rank divison match data set contained many many rows of ```"NaN"``` data. This is most likely due to how I retrieved the data as seen in the "Data Gathering" tab. If the match ```queueId``` was not equal to ```'420'``` then I would add a row of ```"NaN"``` values for that match in my dataset. I realized this was probably the best way as I could simply drop those rows in the future when I clean my dataset. After trying to drop the rows that had ```'NaN'```'s I realized that the output would be an empty data frame. Upon further examination, I noticed that there was a mysterious column labeled as ```'0'``` that contained ```'NaN'```'s for every single row. Which means after using the ```.dropna()``` function I would be essentially dropping every single row. 

Knowing that, I decided to use the ```.drop()``` function and setting ```axis=1``` to drop the column ```'0'``` for every single rank division match data. After this, I attempted the ```.dropna()``` function again on each rank divison data set and fortunately it worked as planned. At a glance, many rows were eliminated this way (on average like 40% of all matches were dropped) but what was also interesting to see is that higher ranked players had less dropped rows. This meant they played more ranked matches recently (when I retrieved the data) than lower ranked players.

In [ ]:
#cleaning each imported dataframe:

#every data set has a column called '0' that turns into NaN after read_csv()
#eliminate that col for each dataframe:
iron_df = iron_df.drop('0', axis=1)
bronze_df = bronze_df.drop('0', axis=1)
silver_df = silver_df.drop('0', axis=1)
gold_df = gold_df.drop('0', axis=1)
platinum_df = platinum_df.drop('0', axis=1)
emerald_df = emerald_df.drop('0', axis=1)
diamond_df = diamond_df.drop('0', axis=1)

#now remove all rows that contain NaN
#leaves us with no empty rows
iron_df = iron_df.dropna()
bronze_df = bronze_df.dropna()
silver_df = silver_df.dropna()
gold_df = gold_df.dropna()
platinum_df = platinum_df.dropna()
emerald_df = emerald_df.dropna()
diamond_df = diamond_df.dropna()

After having a much more cleaned dataset, I planned on changing some text data found in the ```'position'``` column of the dataset. Since all other text data in the data set was lowercased, I decided to lowercase this entire column of text data as well for easier use later on. Again, I did this for every rank division data set.

In [ ]:
#clean the text data found in column 'position' for each division
#lowercase all the strings found in this column
iron_df['position'] = iron_df['position'].str.lower()
bronze_df['position'] = bronze_df['position'].str.lower()
silver_df['position'] = silver_df['position'].str.lower()
gold_df['position'] = gold_df['position'].str.lower()
platinum_df['position'] = platinum_df['position'].str.lower()
emerald_df['position'] = emerald_df['position'].str.lower()
diamond_df['position'] = diamond_df['position'].str.lower()

Finally, I wanted to use ```CountVectorizer``` function from the ```sklearn``` module in order to count the frequency of how many times each ```'position'``` value was seen in all the matches combined. Thus, this required me to combine all the rank division data frames into one big data frame of all the matches and rank divisions. This was done using Panda's .```concat()``` function. Then I used ```CountVectorizer``` and ```.sum()``` on the resulting data frame to create a small dataframe that put the ```'position'``` values as the index and the frequency of each value under one column. While doing this, I noticed that some matches of a player had the ```'position'``` value equal to ```'invalid'``` meaning they most likely disconnected or was away from keyboard (afk) during the whole match. And so I dropped the rows that had a ```'position'``` value of ```'invalid'```.

In [ ]:
#use CountVectorizer to count how often each position occurs in all the matches combined

#first combine all the rank divison match data together:
league_combined = pd.concat([iron_df, bronze_df, silver_df, gold_df, platinum_df, emerald_df, diamond_df])

#drop any rows that have 'invalid' as in the 'position' column
#these matches are most likely due to players being afk during the whole match
league_combined = league_combined[league_combined['position'] != 'invalid']

#now use CountVectorizer
vectorizer = CountVectorizer()
test = vectorizer.fit_transform(league_combined['position'])
df_count_vectorized = pd.DataFrame(test.toarray(), columns=vectorizer.get_feature_names_out())

#now sum the total number of each positions played from all matches
position_sum = df_count_vectorized.sum()

Since I dropped a few more rows after combining, I decided to output the combined data frame into one .csv file as the cleaned data set for the 7 rank division match data sets that I inputted. Additionally, I outputted the small data frame containing the frequency of each 'position' value with their respective values as the index into its own .csv file. Outputted files: __position_sum_cleaned.csv__ and __league_combined_cleaned.csv__.

I then copied my cleaning process for the raw text data retrieved from the Riot API as well. The cleaned .csv file can also be found in the "cleaned_riot_data" folder.

In [ ]:
#write cleaned data into .csv file:
#league_combined.csv + position_sum.csv
league_combined.to_csv('league_combined_cleaned.csv', index = False)
position_sum.to_csv('position_sum_cleaned.csv', index = True)


<br>

__R Cleaning__<br><br>
In this section, I decided to work on cleaning the ```"Challenger"``` ranked players data set that I retrieved through the Riot API as well as detailed in the "Data Gathering" tab. This was mainly done in R and so I decided on working on cleaning the data set in R as well.

While using R I noticed that after retrieving the data, that the data set did not have the same issues seen in my Python data set where there were missing rows of match data. It seems there was less work to be done in this data set.

As such, there was not much work to be done with this data set. I did the same process as before as I retrieved a basically the same match data format as before in Python but this time in R with the top 300 players in North America. In this case, I noticed again that the values in ```'position'``` were uppercase so I lowercased all the values in every row in that column. Also, I noticed that there wasn't a mysterious ```'0'``` column this time so less work was needed in this case too.

I felt that this data set should be combined into the other combined data set for the 7 rank divisions. As such, I deemed it necessary to add a column in the challenger data set to display the appropriate rank ```"Challenger"``` for each row. So I included a new column into the challenger data frame called ```'rank'``` like I did for the other rank divisions.

When writing the dataframes into a .csv file, I felt that the Challenger data set should be made into a separate one on its own as I might perform analysis on just Challenger players. So writing the Challenger data set would make it convenient for me in the future. Additionally, I wrote the combined data set (7 rank divisions + Challenger rank) into a .csv file.Outputted files: __chall_data_cleaned.csv__ and __league_combined_with_chall_cleaned.csv__.

In [ ]:
```{r packages}
#packages
suppressPackageStartupMessages(suppressWarnings(library(tidyverse)))
```


```{r import_chall_data}
#load in challenger data .csv
chall_data <- read.csv("../../data/raw-riot-data/league data/league_chall.csv")
```

```{r clean}
#lowercase all the values in 'position' column
chall_data$position <- tolower(chall_data$position)

#using logical index, keep all rows that don't have 'invalid' in 'position' column
chall_data <- chall_data[!chall_data$position == 'invalid',]

#add a column called 'rank' and put 'challenger' in each row
chall_data$rank <- 'challenger'

```

```{r combine}
#combine with other combined data set
#import combined data set
combined <- read.csv("../../data/cleaned_riot_data/league_combined_cleaned.csv")

#combine both data frames with rbind()
combined_chall <- rbind(combined, chall_data)

```

```{r csv}
#write combined_chall and chall_data into .csv files
write.csv(combined_chall, "league_combined_with_chall_cleaned.csv", row.names = FALSE)
write.csv(chall_data, "chall_data_cleaned.csv", row.names = FALSE)

```